# MyNet

## 1.1. import what u need


In [1]:
import torch     
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import copy
from sklearn.metrics import accuracy_score,f1_score,roc_curve,precision_recall_curve,average_precision_score,auc
from sklearn.metrics import precision_score, recall_score, f1_score,confusion_matrix,matthews_corrcoef,roc_auc_score
import matplotlib.pyplot as plt
import torch.utils.data as Data
import numpy as np
import dataClass as MyDataClass
import pandas as pd

## 1.2. data preprocessing


In [2]:
BATCH_SIZE = 32

#加载数据集并转换成torch.Tensor
tensor = np.load("./final-project1-j1/train.npy")
df = pd.read_csv("./final-project1-j1/train.csv")
label = list(df["label"])
train_tensor = []
for i in range(len(tensor)):
    train_tensor.append([])
    train_tensor[i].append([])
    for j in range(28):
        train_tensor[i][0].append(tensor[i][j*28:(j+1)*28])
train_tensor = torch.Tensor(train_tensor)

train_dataset = MyDataClass.mydataset(train_tensor, label)
train_dataset, development_dataset = torch.utils.data.random_split(train_dataset, [25000, 5000])
tensor = np.load("./final-project1-j1/test.npy")
test_tensor = []
for i in range(len(tensor)):
    test_tensor.append([])
    test_tensor[i].append([])
    for j in range(28):
        test_tensor[i][0].append(tensor[i][j*28:(j+1)*28])
test_dataset = torch.Tensor(test_tensor)
#加载小批次数据，即将数据集中的data分成每组batch_size的小块，shuffle指定是否随机读取
train_loader = Data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)
development_loader = Data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader = Data.DataLoader(dataset=test_dataset,batch_size=BATCH_SIZE,shuffle=False)


In [3]:
#定义网络模型亦即Net 这里定义一个简单的全连接层784->10
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(1, 16,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(16, 64, kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3,padding=1)
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3,padding=1)
        self.conv6 = nn.Conv2d(128, 256, kernel_size=3,padding=1)
        self.linear1 = nn.Linear(256,128)
        self.linear2 = nn.Linear(128,128)
        self.linear3 = nn.Linear(128,10)        
    
    def forward(self,X):
        X = F.relu(F.max_pool2d(self.conv1(X),2,stride=2))
        X = F.relu(self.conv2(X))
        X = F.relu(F.max_pool2d(self.conv3(X),2,stride=2))
        X = F.relu(self.conv4(X))
        X = F.relu(F.max_pool2d(self.conv5(X),2))
        X = F.relu(F.avg_pool2d(self.conv6(X),2))
        X = X.view(-1, 256)
        X = F.relu(self.linear1(X))
        X = F.dropout(X, training=self.training)
        X = F.relu(self.linear2(X))
        X = F.dropout(X, training=self.training)
        return self.linear3(X)

In [4]:
model = Model()#.cuda() #实例化卷积层
loss = nn.CrossEntropyLoss() #损失函数选择，交叉熵函数
optimizer = optim.SGD(model.parameters(),lr = 0.001,momentum=0.9)

## 1.2. Train


In [5]:
losses = [] 
acces = []
eval_losses = []
eval_acces = []
num_epochs = 1
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5,eta_min=4e-08)
for echo in range(num_epochs):
    train_loss = 0   #定义训练损失
    train_acc = 0    #定义训练准确度
    model.train()    #将网络转化为训练模式
    for i,(X,label) in enumerate(train_loader):     #使用枚举函数遍历train_loader
        #X = X.view(-1,784)       #X:[64,1,28,28] -> [64,784]将X向量展平
        X = Variable(X)#.cuda()          #包装tensor用于自动求梯度
        label = Variable(label)#.cuda()
        out = model(X)           #正向传播
        lossvalue = loss(out,label)         #求损失值
        optimizer.zero_grad()       #优化器梯度归零
        lossvalue.backward()    #反向转播，刷新梯度值
        optimizer.step()        #优化器运行一步，注意optimizer搜集的是model的参数
        scheduler.step()
        
        #计算损失
        train_loss += float(lossvalue)      
        #计算精确度
        _,pred = out.max(1)
        num_correct = (pred == label).sum()
        acc = int(num_correct) / X.shape[0]
        train_acc += acc

    losses.append(train_loss / len(train_loader))
    acces.append(train_acc / len(train_loader))
    print("echo:"+' ' +str(echo))
    print("lose:" + ' ' + str(train_loss / len(train_loader)))
    print("accuracy:" + ' '+str(train_acc / len(train_loader)))

echo: 0
lose: 1.8190904720055172
accuracy: 0.2897218670076726


## 1.3. Development

In [13]:
model.eval() #模型转化为评估模式
development_acc = 0
for X,label in development_loader:
    X = Variable(X)#.cuda()
    label = Variable(label)#.cuda()
    developmentout = model(X)
    developmentloss = loss(developmentout,label)

    _, pred = developmentout.max(1)
    num_correct = (pred == label).sum()
    acc = int(num_correct) / X.shape[0]
    development_acc += acc
print("accuracy:" + ' '+str(development_acc / len(development_loader)))

accuracy: 0.8560182225063938


In [14]:
torch.save(model.state_dict(),'mymodule.pt')

## 1.4. Test


In [15]:
test_pred_label = []
model.eval() #模型转化为评估模式
tensor = np.load("./final-project1-j1/test.npy")
print(len(tensor))
print(len(test_dataset))
for X in test_loader:
    X = Variable(X)#.cuda()
    testout = model(X)

    _, pred = testout.max(1)
    test_pred_label += list(pred)
for i, x in enumerate(test_pred_label):
    test_pred_label[i] = int(x)
data = {'image_id':list(range(5000)), 'label':test_pred_label}
df = pd.DataFrame(data,columns=['image_id','label'])
df.to_csv(r'submit.csv',encoding='gbk',index=False)

5000
5000
